# Models Training Status

In this notebook there are scripts to check which models are there and which are missing.

In [1]:
base_path = "/data/tbaldi/checkpoint/"
sizes = ['small', 'baseline', 'large']
batch_sizes = [16, 32, 64, 128, 256, 512, 1024]
learning_rates = [0.1, 0.05, 0.025, 0.0125, 0.00625, 0.003125, 0.0015625]
precisions = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [2]:
missing = []
def get_missing_econ(bs, lr, precision):
    missing = []
    for size in sizes:
        # print(f'BATCH SIZE: {bs} - LEARNING RATE {lr} - SIZE {size} - PRECISION {precision}')
        for exp in range(1, 4):
            file_path = base_path + f'bs{bs}_lr{lr}/ECON_{precision}b/{size}/{size}_emd_{exp}.txt'
            try:
                with open(file_path, 'r') as file:
                    content = file.read()

            except Exception as e:
                if size == 'large':
                    missing.append((bs, lr, precision))
                    continue
    return set(missing)


In [3]:
missing = {}
for bs in batch_sizes:
    
    print(f'=========== {bs} ==========')
    for lr in learning_rates:
        print(f'---------- {lr} ---------')
        for precision in precisions:
            mis = get_missing_econ(bs, lr, precision)
            if mis:
                print(f'{bs} - {lr} - {precision}')
            

=========== 16 ==========
---------- 0.1 ---------
---------- 0.05 ---------
---------- 0.025 ---------
---------- 0.0125 ---------
---------- 0.00625 ---------
---------- 0.003125 ---------
16 - 0.003125 - 2
---------- 0.0015625 ---------
=========== 32 ==========
---------- 0.1 ---------
---------- 0.05 ---------
---------- 0.025 ---------
---------- 0.0125 ---------
---------- 0.00625 ---------
---------- 0.003125 ---------
---------- 0.0015625 ---------
=========== 64 ==========
---------- 0.1 ---------
---------- 0.05 ---------
---------- 0.025 ---------
---------- 0.0125 ---------
---------- 0.00625 ---------
---------- 0.003125 ---------
---------- 0.0015625 ---------
64 - 0.0015625 - 8
=========== 128 ==========
---------- 0.1 ---------
---------- 0.05 ---------
---------- 0.025 ---------
---------- 0.0125 ---------
---------- 0.00625 ---------
---------- 0.003125 ---------
---------- 0.0015625 ---------
=========== 256 ==========
---------- 0.1 ---------
---------- 0.05 ------